<a href="https://colab.research.google.com/github/hiddenntreasure/Har/blob/master/HAR_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
s=pd.Series([1,2,3,4,5])
s=s.as_matrix()
t=[]
o=pd.Series([6,7,8,9,10])
o=o.as_matrix()
t.append(s)
print(t)
t.append(o)
print(t)
t

[array([1, 2, 3, 4, 5])]
[array([1, 2, 3, 4, 5]), array([ 6,  7,  8,  9, 10])]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


[array([1, 2, 3, 4, 5]), array([ 6,  7,  8,  9, 10])]

In [0]:
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

#DATA

In [0]:
DATADIR = '/content/drive/My Drive/Colab Notebooks/CS 10: Human Acitivity Recognition/UCI_HAR_Dataset'

In [0]:
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [0]:
for signal in SIGNALS:
  print(signal)

body_acc_x
body_acc_y
body_acc_z
body_gyro_x
body_gyro_y
body_gyro_z
total_acc_x
total_acc_y
total_acc_z


In [0]:
def _read_csv(filename):
  a=pd.read_csv(filename,delim_whitespace=True,header=None)
  print(a.shape)
  return a

def load_signals(subset):
  signals_data=[]
  for signal in SIGNALS:
    print(signal)
    filename = f'{DATADIR}/{subset}/Inertial Signals/{signal}_{subset}.txt'
    #print(signal)
    signals_data.append(
        _read_csv(filename).as_matrix()

    )

  return np.transpose(signals_data,(1,2,0))

In [0]:
def load_y(subset):
  filename = f'{DATADIR}/{subset}/y_{subset}.txt'
  y= _read_csv(filename)[0]
  y
  return pd.get_dummies(y).as_matrix()

In [0]:
def load_data():
  X_train,X_test = load_signals('train'), load_signals('test')
  y_train,y_test = load_y('train'),load_y('test')
  return X_train,X_test,y_train,y_test

In [0]:
np.random.seed(42) # to avoid thread
import tensorflow as tf
tf.set_random_seed(42)

In [0]:
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads = 1,
    inter_op_parallelism_threads =1
)

In [0]:
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(),config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense,Dropout

In [0]:
epochs = 30
batch_size = 16
n_hidden = 32

In [0]:
# Count the number of classes
def _count_classes(y):
  return len(set(tuple(category) for category in y))

In [0]:
X_train,X_test,y_train,y_test = load_data()

body_acc_x
(7352, 128)
body_acc_y


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


(7352, 128)
body_acc_z
(7352, 128)
body_gyro_x
(7352, 128)
body_gyro_y
(7352, 128)
body_gyro_z
(7352, 128)
total_acc_x
(7352, 128)
total_acc_y
(7352, 128)
total_acc_z
(7352, 128)
body_acc_x
(2947, 128)
body_acc_y
(2947, 128)
body_acc_z
(2947, 128)
body_gyro_x
(2947, 128)
body_gyro_y
(2947, 128)
body_gyro_z
(2947, 128)
total_acc_x
(2947, 128)
total_acc_y
(2947, 128)
total_acc_z
(2947, 128)
(7352, 1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


(2947, 1)


In [0]:
X_train.shape

(7352, 128, 9)

In [0]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(y_train)

## LSTM Model

In [29]:
model = Sequential()
model.add(LSTM(64,input_shape=(timesteps,input_dim),return_sequences=True))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(8))
model.add(Dense(n_classes,activation='sigmoid'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 128, 64)           18944     
_________________________________________________________________
lstm_9 (LSTM)                (None, 128, 32)           12416     
_________________________________________________________________
lstm_10 (LSTM)               (None, 8)                 1312      
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 54        
Total params: 32,726
Trainable params: 32,726
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [31]:
model.fit(X_train,y_train,batch_size=batch_size,epochs=epochs,validation_data=(X_test,y_test))

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 103s 14ms/step - loss: 1.5223 - acc: 0.4025 - val_loss: 1.5745 - val_acc: 0.3688
Epoch 2/30
7352/7352 [==============================] - 100s 14ms/step - loss: 1.2062 - acc: 0.5400 - val_loss: 1.0408 - val_acc: 0.6054
Epoch 3/30
7352/7352 [==============================] - 98s 13ms/step - loss: 0.9095 - acc: 0.6737 - val_loss: 0.9666 - val_acc: 0.6827
Epoch 4/30
7352/7352 [==============================] - 99s 13ms/step - loss: 0.6882 - acc: 0.7501 - val_loss: 0.9588 - val_acc: 0.5769
Epoch 5/30
7352/7352 [==============================] - 99s 13ms/step - loss: 0.6120 - acc: 0.7401 - val_loss: 0.7509 - val_acc: 0.6753
Epoch 6/30
7352/7352 [==============================] - 98s 13ms/step - loss: 0.5426 - acc: 0.7560 - val_loss: 0.7666 - val_acc: 0.7021
Epoch 7/30
7352/7352 [==============================] - 98s 13ms/step - loss: 0.5127 - acc: 0.7605 - val_loss: 0.6631 - val_acc: 0.749

In [0]:
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [26]:
# Confusion Matrix
print(confusion_matrix(y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 535        0  ...                   1                 0
SITTING                  0      489  ...                   0                 0
STANDING                 0      528  ...                   0                 0
WALKING                  0        0  ...                   4                52
WALKING_DOWNSTAIRS       0        0  ...                   0                 4
WALKING_UPSTAIRS         0        0  ...                   1                 5

[6 rows x 6 columns]


In [27]:
score = model.evaluate(X_test, y_test)

2947/2947 [==============================] - 3s 1ms/step


In [28]:
score

[0.7817624219199298, 0.49949100780454697]